<a href="https://colab.research.google.com/github/danialebrat/Stock_Prediction/blob/main/Codes/Stuck_prediction_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler, minmax_scale

In [ ]:
model = load_model("/content/drive/MyDrive/Stock Prediction/Stock_model.h5")

In [ ]:
import numpy as np
 
alpha = 2 / 6
exp = np.array([alpha * (1 - alpha) ** (5 - i) for i in range(1, 6)]) + (1 - alpha) ** 5 / 5
 
def printTransactions(m, k, d, name, owned, prices):
    max_f = None
    min_f = None
 
    for i in range(k):
        f = getCEMA(prices[i]) - np.mean(prices[i])
        if owned[i] > 0 and f > 0 and (max_f is None or f > max_f[0]):
            max_f = (f, '{0} SELL {1}'.format(name[i], owned[i]))
        elif m > prices[i][-1] and d > 0 and f > 0 and (min_f is None or f < min_f[0]):
            min_f = (f, '{0} BUY {1}'.format(name[i], int(m / prices[i][-1])))
 
    if max_f is not None and min_f is not None:
        print(2)
    elif max_f is not None or min_f is not None:
        print(1)
    else:
        print(0)
 
    if max_f is not None:
        print(max_f[1])
    if min_f is not None:
        print(min_f[1])
 
def getCEMA(a):
    return np.dot(exp, a)
 
def readInputAndRun():
    line = input().split()
    m, k, d = float(line[0]), int(line[1]), int(line[2])
    name = []
    owned = []
    prices = []
    for i in range(k):
        line = input().split()
        name.append(line[0])
        owned.append(int(line[1]))
        prices.append(list(map(float, line[2:])))
    printTransactions(m, k, d, name, owned, prices)
 
readInputAndRun()

In [ ]:
def readInputAndRun():
    line = input().split()
    m, k, d = float(line[0]), int(line[1]), int(line[2])
    name = []
    owned = []
    prices = []
    for i in range(k):
        line = input().split()
        name.append(line[0])
        owned.append(int(line[1]))
        prices.append(list(map(float, line[2:])))
    printTransactions(m, k, d, name, owned, prices)

In [ ]:
def printTransactions(m, k, d, name, owned, prices):
    
    """

    INPUT: 

    m - the amount of money you could spend that day.
    k - the number of different stocks available for buying or selling.
    d - the number of remaining days for trading stocks.

    k lines follow, each in the following format: name owned prices
    name - the name of the stock (a string).
    owned - the number of shares you own of that stock.
    prices - 5 space separated numbers representing the stock's price for the last 5 days. last number is the current stock price.

    OUTPUT:
    Output N - number of transactions, 0 if not any
    output N lines containing the name of the stock, BUY or SELL, and the number of shares to buy or sell.
    """
    current_transaction = []
    potential_sale = []
    potential_buy = []

    # Knapstack
    capacity = m
    weights = []
    profits = []
    stocks = []

    

    for i in range(k):

        future_price = predict_future(prices[i])
        ballance = future_price - prices[i][-1]

        if ballance < 0:
          if owned[i] > 0:
            sale = ('{} SELL {}'.format(name[i], owned[i]))
            potential_sale.append(sale)

            
        else:
          weights.append(prices[i][-1])
          profits.append(ballance)
          stocks.append(name)
          
    

    items, values = unbounded_knapsack(weights, profits, capacity)

    for w,s in zip(weights, stocks):
      count = 0
      for item in items: 
        if item == w:
          count += 1
      buy = ('{} BUY {}'.format(s, count))
      potential_buy.append(buy)

    print_output(potential_buy, potential_sale)


    



    

In [2]:
def predict_future(prices):
     
    price_list = []

    segment = prices.reshape((-1, 1)) 
    
    scaler = MinMaxScaler()
    arr = scaler.fit_transform(segment)

    price_list.append(arr)

    future_price = model.predict(price_list)

    future_price = scaler.inverse_transform(future_price)
    future_price = future_price.reshape(prices.shape)

    return arr

In [ ]:
def print_output(potential_buy, potential_sale):

  if len(potential_buy)+len(potential_sale) == 0:
    print(0)

  else:
    number_of_transaction = len(potential_buy)+len(potential_sale)
    print(number_of_transaction)

    if len(potential_buy) > 0:
      for i in potential_buy:
        print(i)
    
    if len(potential_sale) > 0:
      for i in potential_sale:
        print(i)


In [ ]:
def unbounded_knapsack(weights, profits, capacity):
  
    n = len(profits)
    table = [[0] * (capacity + 1) for i in range(n)]

    # When capacity is 0, none of the item can be selected
    for i in range(n):
        table[i][0] = 0

    # When only 1 item is considered, get the profits using that item for every capapcity 'c'
    for c in range(capacity + 1):
        table[0][c] = (c // weights[0]) * profits[0]

    # For each item, check for every possible capacities
    for i in range(1, n):
        for c in range(1, capacity + 1):
            if (weights[i] <= c):
                table[i][c] = max(table[i - 1][c], profits[i] + table[i][c - weights[i]])
            else:
                table[i][c] = table[i - 1][c]

    items = pick_selected_items(table, weights, profits)

    return items

In [ ]:
def pick_selected_items(table, weights, profits):
    items = []
    values = []
    i, c = len(table)-1, len(table[0])-1
    total_profit = table[i][c]
    while (i-1 >= 0 and table[i][c] >= 0):
        if (table[i][c] != table[i - 1][c]):
            items.append(weights[i])
            values.append(profits[i])
            c -= weights[i]
            total_profit -= profits[i]
        else:
            i -= 1

    while (total_profit > 0):
        items.append(weights[0])
        values.append(profits[0])
        total_profit -= profits[0]
    
    return items, values